In [2]:
import requests
import pandas as pd
import time

BASE_URL_CLUBS = "https://compstats.uefa.com/v1/team-ranking"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; ucl-research-bot/1.0)"
}

def safe_int(x):
    try:
        return int(x)
    except (TypeError, ValueError):
        return None

def scrape_stats_group(season_year: int, stats_list, group_name: str,
                       limit: int = 200, offset: int = 0) -> pd.DataFrame:
    """
    Descarga estadísticas de clubes para una temporada y un grupo de stats.
    Se mantiene la misma estructura (season_year, team_id, team_code, team_name_en,
    team_name_es, country_en, country_es) en todos los CSV.
    """
    params = {
        "competitionId": "1",
        "limit": str(limit),
        "offset": str(offset),
        "optionalFields": "PLAYER,TEAM",
        "order": "DESC",
        "phase": "TOURNAMENT",
        "seasonYear": str(season_year),
        "stats": ",".join(stats_list),
    }

    r = requests.get(BASE_URL_CLUBS, params=params, headers=HEADERS, timeout=15)
    r.raise_for_status()
    data = r.json()

    rows = []
    for entry in data:
        team = entry.get("team", {})
        stats_list_resp = entry.get("statistics", [])
        stats_dict = {s.get("name"): s.get("value") for s in stats_list_resp}
        translations = team.get("translations", {})
        display_name = translations.get("displayName", {})
        country_name = translations.get("countryName", {})

        row = {
            "season_year": season_year,
            "team_id": entry.get("teamId"),
            "team_code": team.get("teamCode"),
            "team_name_en": display_name.get("EN"),
            "team_name_es": display_name.get("ES"),
            "country_en": country_name.get("EN"),
            "country_es": country_name.get("ES"),
        }

        # Añadimos SOLO las estadísticas específicas de este grupo
        for stat in stats_list:
            row[f"{group_name}{stat}"] = safe_int(stats_dict.get(stat))

        rows.append(row)

    return pd.DataFrame(rows)


if __name__ == "__main__":
    # 📌 Stats por pestaña que quieres scrapear
    STAT_GROUPS = {
        "key": [
            "matches_appearance",
            "matches_win",
            "matches_draw",
            "matches_loss",
        ],
        "goals": [
            "goals",
            "goals_scored_with_right",
            "goals_scored_with_left",
            "goals_scored_head",
            "goals_scored_inside_penalty_area",
            "goals_scored_outside_penalty_area",
            "penalty_scored",
            "matches_appearance",
        ],
        "attempts": [
            "attempts",
            "attempts_on_target",
            "attempts_off_target",
            "attempts_blocked",
            "matches_appearance",
        ],
        "distribution": [
            "passes_accuracy",
            "passes_attempted",
            "passes_completed",
            "ball_possession",
            "cross_accuruacy",
            "cross_attempted",
            "cross_completed",
            "free_kick",
            "matches_appearance",
        ],
        "attacking": [
            "attacks",
            "assists",
            "corners",
            "offsides",
            "dribbling",
            "matches_appearance",
        ],
        "defending": [
            "recovered_ball",
            "tackles"
            "tackles_won",
            "tackles_lost",
            "clearance_attempted",
            "matches_appearance",
        ],
        "goalkeeping": [
            "saves",
            "goals_conceded",
            "own_goal_conceded",
            "saves_on_penalty",
            "clean_sheet",
            "punches",
            "matches_appearance",
        ],
        "disciplinary": [
            "fouls_committed",
            "fouls_suffered",
            "yellow_cards",
            "red_cards",
            "matches_appearance",
        ],

    }

    for group_name, stats_list in STAT_GROUPS.items():
        print(f"\n📊 Extrayendo estadísticas: {group_name}")
        all_dfs = []

        for season in range(1992, 2026):
            print(f"  ➤ Temporada {season}/{season+1}…")
            try:
                df_season = scrape_stats_group(season, stats_list, group_name)
                if not df_season.empty:
                    all_dfs.append(df_season)
            except Exception as e:
                print(f"    ⚠ Error en temporada {season}: {e}")
            time.sleep(1)

        if all_dfs:
            final_df = pd.concat(all_dfs, ignore_index=True)

            # Ordenamos las columnas para que los datos básicos siempre estén delante
            col_order = [
                "season_year", "team_id", "team_code", "team_name_en",
                "team_name_es", "country_en", "country_es"
            ]
            other_cols = [c for c in final_df.columns if c not in col_order]
            final_df = final_df[col_order + other_cols]

            file_name = f"../data/ucl_clubs_{group_name}_stats_1992_2025.csv"
            final_df.to_csv(file_name, index=False)
            print(f"📁 Guardado correctamente: {file_name}")
        else:
            print(f"❌ No se han generado datos para {group_name}.")


📊 Extrayendo estadísticas: key
  ➤ Temporada 1992/1993…
  ➤ Temporada 1993/1994…
  ➤ Temporada 1994/1995…
  ➤ Temporada 1995/1996…
  ➤ Temporada 1996/1997…
  ➤ Temporada 1997/1998…
  ➤ Temporada 1998/1999…
  ➤ Temporada 1999/2000…
  ➤ Temporada 2000/2001…
  ➤ Temporada 2001/2002…
  ➤ Temporada 2002/2003…
  ➤ Temporada 2003/2004…
  ➤ Temporada 2004/2005…
  ➤ Temporada 2005/2006…
  ➤ Temporada 2006/2007…
  ➤ Temporada 2007/2008…
  ➤ Temporada 2008/2009…
  ➤ Temporada 2009/2010…
  ➤ Temporada 2010/2011…
  ➤ Temporada 2011/2012…
  ➤ Temporada 2012/2013…
  ➤ Temporada 2013/2014…
  ➤ Temporada 2014/2015…
  ➤ Temporada 2015/2016…
  ➤ Temporada 2016/2017…
  ➤ Temporada 2017/2018…
  ➤ Temporada 2018/2019…
  ➤ Temporada 2019/2020…
  ➤ Temporada 2020/2021…
  ➤ Temporada 2021/2022…
  ➤ Temporada 2022/2023…
  ➤ Temporada 2023/2024…
  ➤ Temporada 2024/2025…
  ➤ Temporada 2025/2026…
📁 Guardado correctamente: ../data/ucl_clubs_key_stats_1992_2025.csv

📊 Extrayendo estadísticas: goals
  ➤ Temporada 19

C:\Users\Usuario\AppData\Local\Temp\ipykernel_24556\3809743789.py:154: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_dfs, ignore_index=True)


📁 Guardado correctamente: ../data/ucl_clubs_goals_stats_1992_2025.csv

📊 Extrayendo estadísticas: attempts
  ➤ Temporada 1992/1993…
  ➤ Temporada 1993/1994…
  ➤ Temporada 1994/1995…
  ➤ Temporada 1995/1996…
  ➤ Temporada 1996/1997…
  ➤ Temporada 1997/1998…
  ➤ Temporada 1998/1999…
  ➤ Temporada 1999/2000…
  ➤ Temporada 2000/2001…
  ➤ Temporada 2001/2002…
  ➤ Temporada 2002/2003…
  ➤ Temporada 2003/2004…
  ➤ Temporada 2004/2005…
  ➤ Temporada 2005/2006…
  ➤ Temporada 2006/2007…
  ➤ Temporada 2007/2008…
  ➤ Temporada 2008/2009…
  ➤ Temporada 2009/2010…
  ➤ Temporada 2010/2011…
  ➤ Temporada 2011/2012…
  ➤ Temporada 2012/2013…
  ➤ Temporada 2013/2014…
  ➤ Temporada 2014/2015…
  ➤ Temporada 2015/2016…
  ➤ Temporada 2016/2017…
  ➤ Temporada 2017/2018…
  ➤ Temporada 2018/2019…
  ➤ Temporada 2019/2020…
  ➤ Temporada 2020/2021…
  ➤ Temporada 2021/2022…
  ➤ Temporada 2022/2023…
  ➤ Temporada 2023/2024…
  ➤ Temporada 2024/2025…
  ➤ Temporada 2025/2026…


C:\Users\Usuario\AppData\Local\Temp\ipykernel_24556\3809743789.py:154: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_dfs, ignore_index=True)


📁 Guardado correctamente: ../data/ucl_clubs_attempts_stats_1992_2025.csv

📊 Extrayendo estadísticas: distribution
  ➤ Temporada 1992/1993…
  ➤ Temporada 1993/1994…
  ➤ Temporada 1994/1995…
  ➤ Temporada 1995/1996…
  ➤ Temporada 1996/1997…
  ➤ Temporada 1997/1998…
  ➤ Temporada 1998/1999…
  ➤ Temporada 1999/2000…
  ➤ Temporada 2000/2001…
  ➤ Temporada 2001/2002…
  ➤ Temporada 2002/2003…
  ➤ Temporada 2003/2004…
  ➤ Temporada 2004/2005…
  ➤ Temporada 2005/2006…
  ➤ Temporada 2006/2007…
  ➤ Temporada 2007/2008…
  ➤ Temporada 2008/2009…
  ➤ Temporada 2009/2010…
  ➤ Temporada 2010/2011…
  ➤ Temporada 2011/2012…
  ➤ Temporada 2012/2013…
  ➤ Temporada 2013/2014…
  ➤ Temporada 2014/2015…
  ➤ Temporada 2015/2016…
  ➤ Temporada 2016/2017…
  ➤ Temporada 2017/2018…
  ➤ Temporada 2018/2019…
  ➤ Temporada 2019/2020…
  ➤ Temporada 2020/2021…
  ➤ Temporada 2021/2022…
  ➤ Temporada 2022/2023…
  ➤ Temporada 2023/2024…
  ➤ Temporada 2024/2025…
  ➤ Temporada 2025/2026…


...

📁 Guardado correctamente: ../data/ucl_clubs_distribution_stats_1992_2025.csv

📊 Extrayendo estadísticas: attacking
  ➤ Temporada 1992/1993…
  ➤ Temporada 1993/1994…
  ➤ Temporada 1994/1995…
  ➤ Temporada 1995/1996…
  ➤ Temporada 1996/1997…
  ➤ Temporada 1997/1998…
  ➤ Temporada 1998/1999…
  ➤ Temporada 1999/2000…
  ➤ Temporada 2000/2001…
  ➤ Temporada 2001/2002…
  ➤ Temporada 2002/2003…
  ➤ Temporada 2003/2004…
  ➤ Temporada 2004/2005…
  ➤ Temporada 2005/2006…
  ➤ Temporada 2006/2007…
  ➤ Temporada 2007/2008…
  ➤ Temporada 2008/2009…
  ➤ Temporada 2009/2010…
  ➤ Temporada 2010/2011…
  ➤ Temporada 2011/2012…
  ➤ Temporada 2012/2013…
  ➤ Temporada 2013/2014…
  ➤ Temporada 2014/2015…
  ➤ Temporada 2015/2016…
  ➤ Temporada 2016/2017…
  ➤ Temporada 2017/2018…
  ➤ Temporada 2018/2019…
  ➤ Temporada 2019/2020…
  ➤ Temporada 2020/2021…
  ➤ Temporada 2021/2022…
  ➤ Temporada 2022/2023…
  ➤ Temporada 2023/2024…
  ➤ Temporada 2024/2025…
  ➤ Temporada 2025/2026…


C:\Users\Usuario\AppData\Local\Temp\ipykernel_24556\3809743789.py:154: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_dfs, ignore_index=True)


📁 Guardado correctamente: ../data/ucl_clubs_attacking_stats_1992_2025.csv

📊 Extrayendo estadísticas: defending
  ➤ Temporada 1992/1993…
  ➤ Temporada 1993/1994…
  ➤ Temporada 1994/1995…
  ➤ Temporada 1995/1996…
  ➤ Temporada 1996/1997…
  ➤ Temporada 1997/1998…
  ➤ Temporada 1998/1999…
  ➤ Temporada 1999/2000…
  ➤ Temporada 2000/2001…
  ➤ Temporada 2001/2002…
  ➤ Temporada 2002/2003…
  ➤ Temporada 2003/2004…
  ➤ Temporada 2004/2005…
  ➤ Temporada 2005/2006…
  ➤ Temporada 2006/2007…
  ➤ Temporada 2007/2008…
  ➤ Temporada 2008/2009…
  ➤ Temporada 2009/2010…
  ➤ Temporada 2010/2011…
  ➤ Temporada 2011/2012…
  ➤ Temporada 2012/2013…
  ➤ Temporada 2013/2014…
  ➤ Temporada 2014/2015…
  ➤ Temporada 2015/2016…
  ➤ Temporada 2016/2017…
  ➤ Temporada 2017/2018…
  ➤ Temporada 2018/2019…
  ➤ Temporada 2019/2020…
  ➤ Temporada 2020/2021…
  ➤ Temporada 2021/2022…
  ➤ Temporada 2022/2023…
  ➤ Temporada 2023/2024…
  ➤ Temporada 2024/2025…
  ➤ Temporada 2025/2026…
📁 Guardado correctamente: ../data/ucl_

C:\Users\Usuario\AppData\Local\Temp\ipykernel_24556\3809743789.py:154: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(all_dfs, ignore_index=True)


📁 Guardado correctamente: ../data/ucl_clubs_goalkeeping_stats_1992_2025.csv

📊 Extrayendo estadísticas: disciplinary
  ➤ Temporada 1992/1993…
  ➤ Temporada 1993/1994…
  ➤ Temporada 1994/1995…
  ➤ Temporada 1995/1996…
  ➤ Temporada 1996/1997…
  ➤ Temporada 1997/1998…
  ➤ Temporada 1998/1999…
  ➤ Temporada 1999/2000…
  ➤ Temporada 2000/2001…
  ➤ Temporada 2001/2002…
  ➤ Temporada 2002/2003…
  ➤ Temporada 2003/2004…
  ➤ Temporada 2004/2005…
  ➤ Temporada 2005/2006…
  ➤ Temporada 2006/2007…
  ➤ Temporada 2007/2008…
  ➤ Temporada 2008/2009…
  ➤ Temporada 2009/2010…
  ➤ Temporada 2010/2011…
  ➤ Temporada 2011/2012…
  ➤ Temporada 2012/2013…
  ➤ Temporada 2013/2014…
  ➤ Temporada 2014/2015…
  ➤ Temporada 2015/2016…
  ➤ Temporada 2016/2017…
  ➤ Temporada 2017/2018…
  ➤ Temporada 2018/2019…
  ➤ Temporada 2019/2020…
  ➤ Temporada 2020/2021…
  ➤ Temporada 2021/2022…
  ➤ Temporada 2022/2023…
  ➤ Temporada 2023/2024…
  ➤ Temporada 2024/2025…
  ➤ Temporada 2025/2026…
📁 Guardado correctamente: ../data

In [3]:
import requests
import pandas as pd
import time
import warnings

# Opcional: ocultar el FutureWarning de pandas sobre concat
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="The behavior of DataFrame concatenation with empty or all-NA entries is deprecated.*"
)

BASE_URL_PLAYERS = "https://compstats.uefa.com/v1/player-ranking"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; ucl-research-bot/1.0)"
}


def safe_num(x):
    """Convierte a float o devuelve None si no se puede (sirve para enteros y decimales)."""
    try:
        return float(x)
    except (TypeError, ValueError):
        return None


def scrape_stats_group(season_year: int, stats_list, group_name: str,
                       limit: int = 200, offset: int = 0) -> pd.DataFrame:
    """
    Descarga estadísticas de JUGADORES para una temporada y un grupo de stats.
    Devuelve un DataFrame con info de equipo + jugador + stats del grupo.
    """
    params = {
        "competitionId": "1",             # UCL
        "limit": str(limit),
        "offset": str(offset),
        "optionalFields": "PLAYER,TEAM",
        "order": "DESC",
        "phase": "TOURNAMENT",
        "seasonYear": str(season_year),
        "stats": ",".join(stats_list),
    }

    r = requests.get(BASE_URL_PLAYERS, params=params, headers=HEADERS, timeout=15)
    r.raise_for_status()
    data = r.json()

    rows = []
    for entry in data:
        # ---- Equipo ----
        team = entry.get("team", {}) or {}
        translations = team.get("translations", {}) or {}
        display_name = translations.get("displayName", {}) or {}
        country_name = translations.get("countryName", {}) or {}

        # ---- Jugador ----
        player = entry.get("player", {}) or {}

        row = {
            # Contexto temporada
            "season_year": season_year,

            # Equipo
            "team_id": entry.get("teamId"),
            "team_code": team.get("teamCode"),
            "team_name_en": display_name.get("EN"),
            "team_name_es": display_name.get("ES"),
            "country_en": country_name.get("EN"),
            "country_es": country_name.get("ES"),

            # Jugador - identificadores
            "player_id": player.get("id"),
            "player_name": player.get("internationalName"),

            # Jugador - características
            "player_age": safe_num(player.get("age")),
            "player_birth_date": player.get("birthDate"),
            "player_country_code": player.get("countryCode"),
            "player_birth_country_code": player.get("countryOfBirthCode"),
            "player_gender": player.get("gender"),
            "player_field_position": player.get("fieldPosition"),
            "player_detailed_field_position": player.get("detailedFieldPosition"),

            # Club
            "club_id": player.get("clubId"),
            "club_shirt_name": player.get("clubShirtName"),
            "club_jersey_number": player.get("clubJerseyNumber"),
        }

        # ---- Estadísticas del grupo ----
        stats_list_resp = entry.get("statistics", []) or []
        stats_dict = {s.get("name"): s.get("value") for s in stats_list_resp}

        for stat in stats_list:
            row[f"{group_name}__{stat}"] = safe_num(stats_dict.get(stat))

        rows.append(row)

    return pd.DataFrame(rows)


if __name__ == "__main__":
    # 📌 Stats por pestaña que quieres scrapear (tal y como las llama la API)
    STAT_GROUPS = {
        "key": [
            "minutes_played_official",
            "matches_appearance",
            "goals",
            "assists",
            "distance_covered",
            "top_speed",
        ],
        "goals": [
            "goals",
            "goals_scored_with_right",
            "goals_scored_with_left",
            "goals_scored_head",
            "goals_scored_other",
            "goals_scored_inside_penalty_area",
            "goals_scored_outside_penalty_area",
            "penalty_scored",
            "matches_appearance",
        ],
        "attempts": [
            "attempts",
            "attempts_on_target",
            "attempts_off_target",
            "attempts_blocked",
            "matches_appearance",
        ],
        "distribution": [
            "passes_accuracy",
            "passes_attempted",
            "passes_completed",
            "cross_accuracy",
            "cross_attempted",
            "cross_completed",
            "free_kick",
            "matches_appearance",
        ],
        "attacking": [
            "assists",
            "corners",
            "offsides",
            "dribbling",
            "matches_appearance",
        ],
        "defending": [
            "recovered_ball",
            "tackles",
            "tackles_won",
            "tackles_lost",
            "clearance_attempted",
            "matches_appearance",
        ],
        "goalkeeping": [
            "saves",
            "goals_conceded",
            "saves_on_penalty",
            "clean_sheet",
            "punches",
            "matches_appearance",
        ],
        "disciplinary": [
            "fouls_committed",
            "fouls_suffered",
            "yellow_cards",
            "red_cards",
            "minutes_played_official",
            "matches_appearance",
        ],
    }

    for group_name, stats_list in STAT_GROUPS.items():
        print(f"\n📊 Extrayendo estadísticas: {group_name}")
        all_dfs = []

        for season in range(1992, 2026):
            print(f"  ➤ Temporada {season}/{season+1}…")
            try:
                df_season = scrape_stats_group(season, stats_list, group_name)
                if not df_season.empty:
                    all_dfs.append(df_season)
            except Exception as e:
                print(f"    ⚠️ Error en temporada {season}: {e}")
            time.sleep(1)  # para no freír el endpoint

        # Filtramos DF vacíos y DF con todas las celdas a NaN
        all_dfs = [
            df for df in all_dfs
            if not df.empty and not df.isna().all().all()
        ]

        if all_dfs:
            final_df = pd.concat(all_dfs, ignore_index=True)

            # 👉 De momento NO borramos columnas vacías,
            # para asegurarnos de que no perdemos stats como top_speed o distance_covered.
            # Si luego quieres limpiar, puedes descomentar esto:
            # final_df = final_df.dropna(axis=1, how="all")

            # Ordenamos las columnas para que los datos básicos siempre estén delante
            col_order = [
                # Contexto temporada
                "season_year",

                # Equipo
                "team_id",
                "team_code",
                "team_name_en",
                "team_name_es",
                "country_en",
                "country_es",

                # Jugador - identificadores
                "player_id",
                "player_name",

                # Jugador - características
                "player_age",
                "player_birth_date",
                "player_country_code",
                "player_birth_country_code",
                "player_gender",
                "player_field_position",
                "player_detailed_field_position",

                # Club
                "club_id",
                "club_shirt_name",
                "club_jersey_number",
            ]

            base_cols = [c for c in col_order if c in final_df.columns]
            other_cols = [c for c in final_df.columns if c not in base_cols]
            final_df = final_df[base_cols + other_cols]

            file_name = f"../data/ucl_players_{group_name}_stats_1992_2025.csv"
            final_df.to_csv(file_name, index=False)
            print(f"📁 Guardado correctamente: {file_name}")
        else:
            print(f"❌ No se han generado datos para {group_name}.")


📊 Extrayendo estadísticas: key
  ➤ Temporada 1992/1993…
  ➤ Temporada 1993/1994…
  ➤ Temporada 1994/1995…
  ➤ Temporada 1995/1996…
  ➤ Temporada 1996/1997…
  ➤ Temporada 1997/1998…
  ➤ Temporada 1998/1999…
  ➤ Temporada 1999/2000…
  ➤ Temporada 2000/2001…
  ➤ Temporada 2001/2002…
  ➤ Temporada 2002/2003…
  ➤ Temporada 2003/2004…
  ➤ Temporada 2004/2005…
  ➤ Temporada 2005/2006…
  ➤ Temporada 2006/2007…
  ➤ Temporada 2007/2008…
  ➤ Temporada 2008/2009…
  ➤ Temporada 2009/2010…
  ➤ Temporada 2010/2011…
  ➤ Temporada 2011/2012…
  ➤ Temporada 2012/2013…
  ➤ Temporada 2013/2014…
  ➤ Temporada 2014/2015…
  ➤ Temporada 2015/2016…
  ➤ Temporada 2016/2017…
  ➤ Temporada 2017/2018…
  ➤ Temporada 2018/2019…
  ➤ Temporada 2019/2020…
  ➤ Temporada 2020/2021…
  ➤ Temporada 2021/2022…
  ➤ Temporada 2022/2023…
  ➤ Temporada 2023/2024…
  ➤ Temporada 2024/2025…
  ➤ Temporada 2025/2026…
📁 Guardado correctamente: ../data/ucl_players_key_stats_1992_2025.csv

📊 Extrayendo estadísticas: goals
  ➤ Temporada 